In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,15324
2,Huelva,Confirmados PDIA 14 días,3358
3,Huelva,Tasa PDIA 14 días,"657,4735238662106"
4,Huelva,Confirmados PDIA 7 días,2361
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,67
629,Municipio de Huelva sin especificar,Total Confirmados,250
630,Municipio de Huelva sin especificar,Curados,116


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  15324.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  4265.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 250 personas en los últimos 7 días 

Un positivo PCR cada 168 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,15324.0,2361.0,3358.0,510743.0,462.267716,657.473524,1109.0
Huelva-Costa,9043.0,1561.0,2210.0,288115.0,541.797546,767.054822,739.0
Isla Cristina,1131.0,423.0,576.0,21264.0,1989.277652,2708.803612,230.0
Huelva (capital),4265.0,575.0,855.0,143663.0,400.242234,595.142799,230.0
Condado-Campiña,4277.0,469.0,666.0,155057.0,302.469414,429.519467,201.0
Sierra de Huelva-Andévalo Central,1755.0,264.0,386.0,67571.0,390.700152,571.250980,151.0
Lepe,1213.0,140.0,195.0,27431.0,510.371478,710.874558,69.0
Ayamonte,523.0,134.0,157.0,20946.0,639.740285,749.546453,67.0
Villalba del Alcor,272.0,128.0,163.0,3338.0,3834.631516,4883.163571,59.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Encinasola,87.0,51.0,77.0,1305.0,3908.045977,5900.383142,22.0
Villalba del Alcor,272.0,128.0,163.0,3338.0,3834.631516,4883.163571,59.0
Isla Cristina,1131.0,423.0,576.0,21264.0,1989.277652,2708.803612,230.0
Villanueva de las Cruces,8.0,8.0,8.0,373.0,2144.772118,2144.772118,1.0
Calañas,106.0,48.0,59.0,2791.0,1719.813687,2113.937657,33.0
Castaño del Robledo,6.0,3.0,4.0,223.0,1345.291480,1793.721973,0.0
Berrocal,5.0,5.0,5.0,312.0,1602.564103,1602.564103,NaN
Cerro de Andévalo (El),91.0,26.0,32.0,2364.0,1099.830795,1353.637902,19.0
Palma del Condado (La),688.0,79.0,104.0,10761.0,734.132516,966.452932,21.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Lucena del Puerto,115.0,1.0,4.0,3371.0,29.664788,118.659152,0.0,0.250000
Aroche,78.0,6.0,22.0,3073.0,195.248942,715.912789,3.0,0.272727
Bonares,92.0,7.0,20.0,6058.0,115.549686,330.141961,3.0,0.350000
Cortegana,166.0,14.0,37.0,4666.0,300.042863,792.970424,5.0,0.378378
San Juan del Puerto,199.0,15.0,36.0,9300.0,161.290323,387.096774,5.0,0.416667
Puebla de Guzmán,49.0,10.0,24.0,3073.0,325.414904,780.995770,5.0,0.416667
Almendro (El),27.0,3.0,6.0,826.0,363.196126,726.392252,0.0,0.500000
Paterna del Campo,72.0,1.0,2.0,3509.0,28.498148,56.996295,0.0,0.500000
Cumbres Mayores,21.0,1.0,2.0,1755.0,56.980057,113.960114,0.0,0.500000
